In [1]:
#Проект чат-бот который считает % проверенных домашних заданий педагогами и отправляет автоматическое сообщение педагогу, если % проверки ниже 75%

In [1]:
#Библеотека для работы с чат-ботом телеграм
import telebot
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import threading
import time
from datetime import datetime

# Установите свои учетные данные для Google Sheets
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

# Замените на путь к вашему файлу credentials.json
CREDENTIALS_FILE = "C:/Users/1/Desktop/Project (Bot)/projectbot-447014-437b4c89d4a6.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, SCOPE)

# Подключение к Google Sheets
client = gspread.authorize(credentials)
spreadsheet_name = "Отчёт по дз"
sheet = client.open(spreadsheet_name).sheet1

# Создание бота
bot = telebot.TeleBot('7809768765:AAHOm3QIEj-TEE-XTWZ9_Y8R0Tn0rzK8hwk')

# Ваши Chat ID
MY_CHAT_ID = 699343727  # Ваш Chat ID
MK_CHAT_ID = 366888178  # Другой Chat ID
JULIACHERNOVA_CHAT_ID = 1831617416  # Ещё один Chat ID

def calculate_report_for_teacher(df, teacher_id):
    """
    Формирует отчёт только для одного преподавателя по его ID.
    """
    try:
        # Фильтрация данных для конкретного преподавателя
        teacher_data = df[df["ID"] == str(teacher_id)]
        if teacher_data.empty:
            return f"Для преподавателя с ID {teacher_id} данных нет."

        # Формирование отчёта
        for _, row in teacher_data.iterrows():
            full_name = row['ФИО преподавателя'].split()
            if len(full_name) >= 2:
                name = full_name[0]
                patronymic = full_name[1]
            else:
                name = full_name[0]
                patronymic = ""

            выдано = int(row['Выдано']) if not pd.isna(row['Выдано']) else 0
            проверено = int(row['Проверено']) if not pd.isna(row['Проверено']) else 0
            процент_проверки = int(row['Процент проверки']) if not pd.isna(row['Процент проверки']) else 0
            осталось_проверить = выдано - проверено

            if выдано == 0:
                return f"{name} {patronymic}, добрый день! У вас нет выданных заданий для проверки."

            return (
                f"{name} {patronymic}, добрый день! Напоминаю, что вам необходимо проверить ещё {осталось_проверить} заданий.\n"
                f"Вам было отправлено {выдано} заданий, вы проверили {проверено}, что составляет {процент_проверки}%."
            )
    except Exception as e:
        return f"Ошибка при формировании отчёта для ID {teacher_id}: {e}"

def send_daily_reports():
    """
    Ежедневная отправка отчётов каждому преподавателю по его ID.
    """
    target_time = "15:50"  # Время отправки (часы:минуты)
    while True:
        now = datetime.now().strftime("%H:%M")
        if now == target_time:
            try:
                # Получение данных из Google Sheets
                data = sheet.get_all_values()
                headers = data[0]
                rows = data[1:]

                # Создание DataFrame из данных
                df = pd.DataFrame(rows, columns=headers)

                # Преобразование числовых данных в int
                df["Выдано"] = pd.to_numeric(df["Выдано"], errors="coerce")
                df["Проверено"] = pd.to_numeric(df["Проверено"], errors="coerce")
                df["Процент проверки"] = (df["Проверено"] / df["Выдано"] * 100).fillna(0).astype(int)

                # Уникальные ID преподавателей
                teacher_ids = df["ID"].unique()

                for teacher_id in teacher_ids:
                    report_message = calculate_report_for_teacher(df, teacher_id)
                    # Отправка отчёта преподавателю
                    try:
                        bot.send_message(teacher_id, report_message, parse_mode="Markdown")
                        print(f"[{datetime.now()}] Отчёт отправлен преподавателю с ID {teacher_id}")
                    except Exception as e:
                        print(f"[{datetime.now()}] Ошибка при отправке отчёта преподавателю с ID {teacher_id}: {e}")
            except Exception as e:
                print(f"[{datetime.now()}] Ошибка при формировании отчётов: {e}")

            # Ждём 60 секунд, чтобы избежать повторной отправки
            time.sleep(60)
        else:
            # Проверяем каждую секунду
            time.sleep(1)

# Создание и запуск потока для ежедневной отправки отчётов
daily_report_thread = threading.Thread(target=send_daily_reports, daemon=True)
daily_report_thread.start()


# Создание и запуск потока
thread = threading.Thread(target=send_daily_reports, daemon=True)
thread.start()

@bot.message_handler(commands=['help'])
def help(message):
    bot.send_message(message.chat.id, f'Данный чат-бот предназначен для расчёта процента проверенных домашних заданий педагогами и отправляет автоматическое сообщение педагогу, если % проверки ниже 75%')

# Запуск бота
print("Бот запущен. Поток для автоматической отправки сообщений работает...")
bot.polling(none_stop=True)

Бот запущен. Поток для автоматической отправки сообщений работает...
[2025-01-23 15:50:00.718913] Ошибка при отправке отчёта преподавателю с ID : A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: chat_id is empty
[2025-01-23 15:50:00.826879] Отчёт отправлен преподавателю с ID 1831617416
[2025-01-23 15:50:00.912851] Отчёт отправлен преподавателю с ID 699343727
[2025-01-23 15:50:01.026894] Отчёт отправлен преподавателю с ID 366888178
[2025-01-23 15:50:01.266007] Ошибка при отправке отчёта преподавателю с ID : A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: chat_id is empty
[2025-01-23 15:50:01.366973] Отчёт отправлен преподавателю с ID 1831617416
[2025-01-23 15:50:01.456962] Отчёт отправлен преподавателю с ID 699343727
[2025-01-23 15:50:01.542917] Отчёт отправлен преподавателю с ID 366888178
